# 本セクションの目次
1. ビッグデータ世界のDDL
2. テンポラリテーブル
3. ビッグデータ世界のSQLとは？
4. 単純SQLを振り返ってみよう
5. 分析関数を練習してみよう
6. LAG/Lead関数
7. ピボットテーブル
8. SparkのRDDを使って1レコードつづ処理してみよう

In [1]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init("/home/pyspark/spark")

In [2]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .config("spark.jars.packages", "org.apache.spark:spark-streaming_2.13:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-avro_2.12:3.2.0") \
    .enableHiveSupport() \
    .getOrCreate()

# パッケージを複数渡したい時は「,」で繋いで渡します。
# Sparkのバージョンにしっかりと合わせます(今回はSparkのバージョンが3.2を使っています。)。

:: loading settings :: url = jar:file:/home/pyspark/spark-3.2.0-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/pyspark/.ivy2/cache
The jars for the packages stored in: /home/pyspark/.ivy2/jars
org.apache.spark#spark-streaming_2.13 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d7a4ee0f-98b7-4414-8aac-31942ffc2fab;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in ce

# ビッグデータの世界のDDL

ビッグデータの世界でのDDLはRDSと同じ様にDDL文を実行することが可能です。  
今回は以下のDDLについてみていきましょう  

- Create Database 文
- CREATE EXTERNAL TABLE文
- CREATE VIEW
- ADD PARTITION(MSCK REPAIR)

## CREATE DATABASE文
データベースの作成を行います。
こちらはRDSなどのCreate Database　と同じ方法で作成が可能です。

In [3]:
spark.sql("create database if not exists super_crush_course")

22/01/07 04:05:07 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
22/01/07 04:05:07 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
Fri Jan 07 04:05:08 UTC 2022 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for server certificate verification.
Fri Jan 07 04:05:08 UTC 2022 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not us

DataFrame[]

In [4]:
# データベースの一覧を見てみましょう
spark.sql("show databases").show(truncate=False)

+----------------------------+
|namespace                   |
+----------------------------+
|data_management_crush_course|
|default                     |
|metadata_tmp                |
|super_crush_course          |
+----------------------------+



## CREATE EXTERNAL TABLE文
テーブル定義の構成要素をみていきましょう

```
CREATE EXTERNAL TABLE IF NOT EXISTS super_crush_course.csv_table ( id INT, date STRING)
PARTITIONED BY (dt INT)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LOCATION '/home/pyspark/super_crush_course.db/csv_table/dataset/parquet/';

#S3などであれば、以下のように設定を変えることも可能です。
LOCATION 's3://data.platform/super_crush_course.db/raw_zone/sampletable/';

```

ビッグデータの世界では、実データとデータベース定義/テーブル定義(メタデータ)は明確に分離されています。  
今回のコースだと実データはローカル端末のSSD(HDD)でテーブル定義はMysqlに登録されています。  
明確に分離されているからこそ、場所を指し示す宣言であるLocationが必要になってきます  

ロケーションは「super_crush_course.db」とDB名.db/TABLE名とすることが通例です。  
Externalは外部のという意味で、オンプレ環境の場合はつけないことが多かったが、クラウド環境ではつけるのが必須となっている設定。

メタデータについてさらに詳しく知りたい方は、以下の講座を参照してみてください  
「【PythonとSparkで始めるデータマネジメント入門】 ビッグデータレイクのための統合メタデータ管理入門」

上記の例は、CSV形式のテーブルです。  
それ以外にもParquet形式、Avro形式でテーブルを作成することができます。

```
# パーティションつきのテーブル
CREATE TABLE IF NOT EXISTS super_crush_course.parquet_table 
(code String, gengo String,wareki String,seireki String,chu String,sokei String,jinko_male String,jinko_femail String)
PARTITIONED BY (kenmei String)
STORED AS PARQUET
TBLPROPERTIES ("parquet.compression"="SNAPPY");
LOCATION '/home/pyspark/super_crush_course.db/parquet_table';

```

圧縮形式と保存するファイルフォーマットを指定してテーブルの作成を行なっていきます。  
ちなみにテーブル定義のカラムは、今回読み込んだCSV/JSONのスキーマになります。  

```
# パーティションなしのテーブルの場合
CREATE TABLE IF NOT EXISTS super_crush_course.parquet_table_with_no_partition
(code String, gengo String,kenmei, String,wareki String,seireki String,chu String,sokei String,jinko_male String,jinko_femail String)
STORED AS PARQUET
TBLPROPERTIES ('parquet.compression'='SNAPPY')
LOCATION '/home/pyspark/super_crush_course.db/parquet_table_with_no_partition'

```

In [9]:
## テーブル作成
# テーブル作成も同様に、spark.sqlを使ってテーブルを作成していきます。
# ロケーションはセクション2で出力したディレクトリになります。

# Parquetテーブルの作成(パーティションあり)
spark.sql("""

CREATE TABLE IF NOT EXISTS super_crush_course.parquet_table_with_partition 
(code String, gengo String,wareki String,seireki String,chu String,sokei String,jinko_male String,jinko_femail String)
PARTITIONED BY (kenmei String)
STORED AS PARQUET
TBLPROPERTIES ('parquet.compression'='SNAPPY')
LOCATION '/home/pyspark/super_crush_course.db/parquet_table_with_partition'

""")

# Paruqetテーブルの作成(パーティションなし)
spark.sql("""

CREATE TABLE IF NOT EXISTS super_crush_course.parquet_table_with_no_partition 
(code String, gengo String,wareki String,kenmei String,seireki String,chu String,sokei String,jinko_male String,jinko_femail String)
STORED AS PARQUET
TBLPROPERTIES ('parquet.compression'='SNAPPY')
LOCATION '/home/pyspark/super_crush_course.db/parquet_table_with_no_partition'

""")

DataFrame[]

In [18]:
# AVROテーブルの作成(パーティションあり)
spark.sql("""

CREATE TABLE IF NOT EXISTS super_crush_course.avro_table_with_partition 
(code String, gengo String,wareki String,seireki String,chu String,sokei String,jinko_male String,jinko_femail String)
PARTITIONED BY (kenmei String)
STORED AS AVRO
TBLPROPERTIES ('avro.compression'='SNAPPY')
LOCATION '/home/pyspark/super_crush_course.db/parquet_table_with_partition'

""")

# AVROテーブルの作成(パーティションなし)
spark.sql("""

CREATE TABLE IF NOT EXISTS super_crush_course.avro_table_with_no_partition 
(code String, gengo String,wareki String,kenmei String,seireki String,chu String,sokei String,jinko_male String,jinko_femail String)
STORED AS AVRO
TBLPROPERTIES ('avro.compression'='SNAPPY')
LOCATION '/home/pyspark/super_crush_course.db/avro_table_with_no_partition'

""")

DataFrame[]

In [17]:
# 作成したテーブルを見てみましょう
spark.sql("show tables in super_crush_course").show(truncate=False)

+------------------+-------------------------------+-----------+
|namespace         |tableName                      |isTemporary|
+------------------+-------------------------------+-----------+
|super_crush_course|avro_table_with_no_partition   |false      |
|super_crush_course|ctas_dataframe                 |false      |
|super_crush_course|ctas_sql                       |false      |
|super_crush_course|parquet_table_with_no_partition|false      |
|super_crush_course|parquet_table_with_partition   |false      |
+------------------+-------------------------------+-----------+



# ADD PRTITION
パーティションを認識するためにコマンドを発行する必要があります。

- Add Partiton
- MSCK repair Table

の2つをみていきましょう

In [10]:
# パーティションも管理されている
spark.sql("show partitions super_crush_course.parquet_table_with_partition").show(n=2)
# 本来kenmeiがあってほしいが。。。

+---------+
|partition|
+---------+
+---------+



In [11]:
# パーティションなしテーブルは。。？クエリしてみる
spark.sql("select * from super_crush_course.parquet_table_with_no_partition").show(n=2)

+----+-----+------+------+-------+----+-------+----------+------------+
|code|gengo|wareki|kenmei|seireki| chu|  sokei|jinko_male|jinko_femail|
+----+-----+------+------+-------+----+-------+----------+------------+
|  25| 平成|    22|滋賀県|   2010|null|1410777|    696769|        null|
|  27| 昭和|    10|大阪府|   1935|null|4297174|   2241666|        null|
+----+-----+------+------+-------+----+-------+----------+------------+
only showing top 2 rows



In [12]:
# パーティションがあってadd paritionをしていないテーブルは。。？クエリしてみる
spark.sql("select * from super_crush_course.parquet_table_with_partition").show(n=2)
# データを見ることができない

+----+-----+------+-------+---+-----+----------+------------+------+
|code|gengo|wareki|seireki|chu|sokei|jinko_male|jinko_femail|kenmei|
+----+-----+------+-------+---+-----+----------+------------+------+
+----+-----+------+-------+---+-----+----------+------------+------+



In [13]:
# パーティションを追加してみる
# パーティションの追加には２種類存在しています
# add partition
# msck repair table名
spark.sql("alter table super_crush_course.parquet_table_with_partition add if not exists  partition (kenmei='東京都')")

22/01/07 04:17:07 WARN log: Updating partition stats fast for: parquet_table_with_partition
22/01/07 04:17:07 WARN log: Updated size to 5095


DataFrame[]

In [20]:
# パーティションも管理されている
spark.sql("show partitions super_crush_course.parquet_table_with_partition").show(n=20)

+-----------------------------+
|                    partition|
+-----------------------------+
|         kenmei=__HIVE_DEF...|
|                kenmei=三重県|
|                kenmei=京都府|
|          kenmei=人口集中地区|
|kenmei=人口集中地区以外の地区|
|                kenmei=佐賀県|
|                  kenmei=全国|
|                kenmei=兵庫県|
|                kenmei=北海道|
|                kenmei=千葉県|
|              kenmei=和歌山県|
|                kenmei=埼玉県|
|                kenmei=大分県|
|                kenmei=大阪府|
|                kenmei=奈良県|
|                kenmei=宮城県|
|                kenmei=宮崎県|
|                kenmei=富山県|
|                kenmei=山口県|
|                kenmei=山形県|
+-----------------------------+
only showing top 20 rows



In [15]:
# 再度検索を行ってみる
# パーティションがあってadd paritionをしていないテーブルは。。？クエリしてみる
spark.sql("select * from super_crush_course.parquet_table_with_partition").show()
# 追加した東京都のデータだけ見える

+----+-----+------+-------+----+--------+----------+------------+------+
|code|gengo|wareki|seireki| chu|   sokei|jinko_male|jinko_femail|kenmei|
+----+-----+------+-------+----+--------+----------+------------+------+
|  13| 平成|    17|   2005|null|12576601|   6264895|        null|東京都|
|  13| 昭和|    55|   1980|null|11618281|   5856280|        null|東京都|
|  13| 平成|     7|   1995|null|11773605|   5892704|        null|東京都|
|  13| 昭和|    25|   1950|null| 6277500|   3169389|        null|東京都|
|  13| 大正|     9|   1920|null| 3699428|   1952989|        null|東京都|
|  13| 昭和|     5|   1930|null| 5408678|   2855323|        null|東京都|
|  13| 平成|    27|   2015|null|13515271|   6666690|        null|東京都|
|  13| 平成|     2|   1990|null|11855563|   5969773|        null|東京都|
|  13| 平成|    22|   2010|null|13159388|   6512110|        null|東京都|
|  13| 昭和|    60|   1985|null|11829363|   5955029|        null|東京都|
|  13| 大正|    14|   1925|null| 4485144|   2387609|        null|東京都|
|  13| 昭和|    30|   1955|null| 80

In [19]:
# 一個づつAdd partitionするのは面倒なのでmsckを使う(ただし時間がかかる場合が多いので、日々の処理であればadd partitionを選択する方がいい)
spark.sql("msck repair table super_crush_course.parquet_table_with_partition")
spark.sql("msck repair table super_crush_course.avro_table_with_partition")

DataFrame[]

# Create View
ビューを作成します。
ビューとは、仮想的なテーブルのことでデータを生成しなくてもテーブルを生成することが可能です。

言葉で伝えるより実際に見た方がいいと思うので、早速作ってみましょう。　　

手っ取り早くクエリを簡単にしたい場合に有効です。

In [23]:
# create view

spark.sql("""

create view if not exists parquet_view (gengo)
as 
select gengo from 
super_crush_course.parquet_table_with_partition

""")

DataFrame[]

In [24]:
spark.sql("select * from parquet_view").show()

+-----+
|gengo|
+-----+
| null|
| 昭和|
| 昭和|
| 平成|
| 昭和|
| 昭和|
| 昭和|
| 昭和|
| 昭和|
| 大正|
| 昭和|
| 昭和|
| 平成|
| 平成|
| 昭和|
| 昭和|
| 昭和|
| 平成|
| 平成|
| 昭和|
+-----+
only showing top 20 rows



# テンポラリテーブル
テンポラリーテーブルとはデータフレームから一時的にテーブルを作成することで、SparkSessionごとに生成が可能です。

特にスキーマオンリードで読み込んだdataframeを一時的にテーブルにすることで、SQLでの操作を可能にすることができます。

一連の流れを見てみましょう

In [25]:
# テンポラリーテーブルの作成
json_df=spark.read.json("./dataset/jinko.json")
json_df.createOrReplaceTempView("json_tmp")

In [26]:
#　json_tempの検索
spark.sql("select * from json_tmp where kenmei='東京都'").show()

+----+----+-----+------------+----------+------+-------+--------+------+
| chu|code|gengo|jinko_female|jinko_male|kenmei|seireki|   sokei|wareki|
+----+----+-----+------------+----------+------+-------+--------+------+
|null|  13| 大正|     1746439|   1952989|東京都|   1920| 3699428|     9|
|null|  13| 大正|     2097535|   2387609|東京都|   1925| 4485144|    14|
|null|  13| 昭和|     2553355|   2855323|東京都|   1930| 5408678|     5|
|null|  13| 昭和|     3044223|   3325696|東京都|   1935| 6369919|    10|
|null|  13| 昭和|     3559096|   3795875|東京都|   1940| 7354971|    15|
|  1)|  13| 昭和|     1700139|   1788145|東京都|   1945| 3488284|    20|
|null|  13| 昭和|     3108111|   3169389|東京都|   1950| 6277500|    25|
|null|  13| 昭和|     3921261|   4115823|東京都|   1955| 8037084|    30|
|  2)|  13| 昭和|     4686779|   4997023|東京都|   1960| 9683802|    35|
|null|  13| 昭和|     5304661|   5564583|東京都|   1965|10869244|    40|
|null|  13| 昭和|     5607062|   5801009|東京都|   1970|11408071|    45|
|null|  13| 昭和|     5760181|   59

# ビッグデータ世界のDMLとは

ビッグデータの世界のSQLは基本的にSQL’ライク’です。  
というのもRDSのSQLを前提にしてライクと言っているだけなので、ビッグデータ世界を中心としたらSQLそのものです。  
RDSでのSQLに慣れている人は、ビッグデータの世界のSQLは難なくこなすことができると思います。  

- SELECT
- CTAS
- SELECT INSERT
- INSERT?
- UPDATE?
- DELETE?

今回はこれらのDMLを確認してみます。

In [ ]:
# SELECT
spark.sql("select * from super_crush_course.parquet_table_with_no_partition where kenmei ='東京都'").show()

## パーティションつきのテーブルを検索してみる
spark.sql("select * from super_crush_course.parquet_table_with_partition where kenmei ='東京都'").show()

# パーティションなしの場合は、すべてのデータを走査してから絞り込みます
# パーティションありの場合は、特定のパーティション配下のディレクトリのみをチェックします

# 大体のRDSのSQLでできることは実行可能です。

In [ ]:
# Joinも可能
# SQLでJoinするというのはRDBを使っている方ならイメージが湧くと思いますので、今回はデータフレームでジョインをしてみたいと思います
json_df.join(json_df,on=[json_df.code == json_df.code,json_df.wareki == json_df.wareki],how="inner").show()

# CTAS
Create Table As Selectの略です。

簡単にいうと、Selectの返却結果からテーブルを作成することが可能です。

In [27]:
# CTASを動かしてみます

# SQLでやる方法
spark.sql(""" 
CREATE EXTERNAL TABLE if not exists super_crush_course.ctas_sql 
    STORED AS PARQUET LOCATION '/home/pyspark/super_crush_course.db/ctas_sql' 
AS
SELECT *
    FROM super_crush_course.parquet_table_with_no_partition
WHERE 1=1

""")

DataFrame[]

In [28]:
spark.sql("show tables in super_crush_course").show(truncate=False)

+------------------+-------------------------------+-----------+
|namespace         |tableName                      |isTemporary|
+------------------+-------------------------------+-----------+
|super_crush_course|avro_table_with_no_partition   |false      |
|super_crush_course|avro_table_with_partition      |false      |
|super_crush_course|ctas_dataframe                 |false      |
|super_crush_course|ctas_sql                       |false      |
|super_crush_course|parquet_table_with_no_partition|false      |
|super_crush_course|parquet_table_with_partition   |false      |
|                  |json_tmp                       |true       |
+------------------+-------------------------------+-----------+



In [29]:
json_df.write.format("parquet").mode("overwrite").saveAsTable("super_crush_course.ctas_dataframe",path='/home/pyspark/super_crush_course.db/ctas_dataframe')

22/01/07 04:28:45 WARN HadoopFSUtils: The directory file:/home/pyspark/super_crush_course.db/ctas_dataframe was not found. Was it deleted very recently?


In [30]:
spark.sql("select * from super_crush_course.ctas_dataframe").printSchema()
spark.sql("show create table super_crush_course.ctas_dataframe").show(truncate=False)

root
 |-- chu: string (nullable = true)
 |-- code: string (nullable = true)
 |-- gengo: string (nullable = true)
 |-- jinko_female: string (nullable = true)
 |-- jinko_male: string (nullable = true)
 |-- kenmei: string (nullable = true)
 |-- seireki: string (nullable = true)
 |-- sokei: string (nullable = true)
 |-- wareki: string (nullable = true)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|createtab_stmt                                                                                                                                                                                                                                                                                                                   |


In [31]:
spark.sql("select * from super_crush_course.ctas_dataframe").show()

+----+----+-----+------------+----------+--------+-------+--------+------+
| chu|code|gengo|jinko_female|jinko_male|  kenmei|seireki|   sokei|wareki|
+----+----+-----+------------+----------+--------+-------+--------+------+
|null|  00| 大正|    27918868|  28044185|    全国|   1920|55963053|     9|
|null|  01| 大正|     1114861|   1244322|  北海道|   1920| 2359183|     9|
|null|  02| 大正|      375161|    381293|  青森県|   1920|  756454|     9|
|null|  03| 大正|      424471|    421069|  岩手県|   1920|  845540|     9|
|null|  04| 大正|      476459|    485309|  宮城県|   1920|  961768|     9|
|null|  05| 大正|      444855|    453682|  秋田県|   1920|  898537|     9|
|null|  06| 大正|      490597|    478328|  山形県|   1920|  968925|     9|
|null|  07| 大正|      689225|    673525|  福島県|   1920| 1362750|     9|
|null|  08| 大正|      688272|    662128|  茨城県|   1920| 1350400|     9|
|null|  09| 大正|      532224|    514255|  栃木県|   1920| 1046479|     9|
|null|  10| 大正|      538504|    514106|  群馬県|   1920| 1052610|     9|
|nul

# SELECT INSERT
既に存在するテーブルに対して、検索結果をもとにデータを登録していくことができます。  
SELECT INSERTの場合は、ADD PARTITIONは不要です

CTASと違うのはこちらはテーブルを作る操作ではなくて既にあるテーブルに対してデータを登録することが目的です。

In [32]:
# 今回は各地域のデータの履歴をまとめて一つのパーティションに入れるSelect Insertを記載してましょう。
spark.sql(""" 
Insert overwrite table super_crush_course.parquet_table_with_partition PARTITION(kenmei='all')

select code,gengo,wareki,seireki,chu,sokei,jinko_male,jinko_femail
 from 
super_crush_course.parquet_table_with_no_partition

""")

22/01/07 04:31:19 WARN log: Updating partition stats fast for: parquet_table_with_partition
22/01/07 04:31:19 WARN log: Updated size to 22841


DataFrame[]

In [33]:
spark.sql("show partitions super_crush_course.parquet_table_with_partition").show()

+-----------------------------+
|                    partition|
+-----------------------------+
|         kenmei=__HIVE_DEF...|
|                   kenmei=all|
|                kenmei=三重県|
|                kenmei=京都府|
|          kenmei=人口集中地区|
|kenmei=人口集中地区以外の地区|
|                kenmei=佐賀県|
|                  kenmei=全国|
|                kenmei=兵庫県|
|                kenmei=北海道|
|                kenmei=千葉県|
|              kenmei=和歌山県|
|                kenmei=埼玉県|
|                kenmei=大分県|
|                kenmei=大阪府|
|                kenmei=奈良県|
|                kenmei=宮城県|
|                kenmei=宮崎県|
|                kenmei=富山県|
|                kenmei=山口県|
+-----------------------------+
only showing top 20 rows



# INSERT/UPDATE/DELETE?
ビッグデータの世界では原則としてACIDをサポートしていません。  
そのため、UPDATEやDELETがサポートされていないことが多いです。

INSERTは単体で利用することはできますが、あまり出番がなく前述で紹介したSELECT INSERTでの出番が大半です。

# 分析関数の練習をしよう
ここからは、分析関数の練習をしてみましょう。  

- agg(groupby,count,sum)
- window(over)
- ピボットテーブル
- lag/lead関数

データフレームでの操作とSQLでの操作を対比させながら実行していきます。

# 利用するテーブル(とDataFrame)
spark.sql("select * from super_crush_course.parquet_table_with_no_partition")

In [34]:
analysis_df=spark.sql("select * from super_crush_course.parquet_table_with_no_partition")

In [35]:
from pyspark.sql import functions as F 
# groupby->agg
# groupbyしたそれぞれのグループの中でデータを溜め込んでいく(count,sum,avg)こと

analysis_df.groupBy(analysis_df.code).count().show(n=2)

analysis_df.groupBy(analysis_df.code).agg(F.sum(analysis_df.jinko_male).alias("sum_male"),F.avg(analysis_df.jinko_male).alias("avg_me")).show(n=2) 

+----+-----+
|code|count|
+----+-----+
|  07|   20|
|  15|   20|
+----+-----+
only showing top 2 rows

+----+-----------+---------+
|code|   sum_male|   avg_me|
+----+-----------+---------+
|  07| 1.852108E7| 926054.0|
|  15|2.2317954E7|1115897.7|
+----+-----------+---------+
only showing top 2 rows



In [36]:
# sqlで表現してみましょう
spark.sql("""
select code,sum(jinko_male) as sum_male,avg(jinko_male) as avg_me from super_crush_course.parquet_table_with_no_partition
group by code
""").show(n=2)

+----+-----------+---------+
|code|   sum_male|   avg_me|
+----+-----------+---------+
|  07| 1.852108E7| 926054.0|
|  15|2.2317954E7|1115897.7|
+----+-----------+---------+
only showing top 2 rows



In [37]:
from pyspark.sql.window import Window

# window 関数
# window->{paritionBy(=GroupBy)->(order by) -> rank(順番付け)}
# 最後にwindowをoverにいれると結果が出ます

window_schema = Window.partitionBy(analysis_df.code).orderBy(analysis_df.sokei.cast("Long").asc())
analysis_df.withColumn("sokei_rank",F.rank().over(window_schema)).show(n=50)

+----+-----+------+------+-------+----+---------+----------+------------+----------+
|code|gengo|wareki|kenmei|seireki| chu|    sokei|jinko_male|jinko_femail|sokei_rank|
+----+-----+------+------+-------+----+---------+----------+------------+----------+
|  00| 大正|     9|  全国|   1920|null| 55963053|  28044185|        null|         1|
|  00| 大正|    14|  全国|   1925|null| 59736822|  30013109|        null|         2|
|  00| 昭和|     5|  全国|   1930|null| 64450005|  32390155|        null|         3|
|  00| 昭和|    10|  全国|   1935|null| 69254148|  34734133|        null|         4|
|  00| 昭和|    20|  全国|   1945|  1)| 71998104|  33894059|        null|         5|
|  00| 昭和|    15|  全国|   1940|null| 73114308|  36566010|        null|         6|
|  00| 昭和|    25|  全国|   1950|null| 84114574|  41241192|        null|         7|
|  00| 昭和|    30|  全国|   1955|null| 90076594|  44242657|        null|         8|
|  00| 昭和|    35|  全国|   1960|  2)| 94301623|  46300445|        null|         9|
|  00| 昭和|    40

In [38]:
# SQLで表現してみましょう

spark.sql("""
select 
*,
RANK() OVER (PARTITION BY code ORDER BY cast(sokei as long) asc) as sokei_rank

from super_crush_course.parquet_table_with_no_partition 
""").show(n=50)

+----+-----+------+------+-------+----+---------+----------+------------+----------+
|code|gengo|wareki|kenmei|seireki| chu|    sokei|jinko_male|jinko_femail|sokei_rank|
+----+-----+------+------+-------+----+---------+----------+------------+----------+
|  00| 大正|     9|  全国|   1920|null| 55963053|  28044185|        null|         1|
|  00| 大正|    14|  全国|   1925|null| 59736822|  30013109|        null|         2|
|  00| 昭和|     5|  全国|   1930|null| 64450005|  32390155|        null|         3|
|  00| 昭和|    10|  全国|   1935|null| 69254148|  34734133|        null|         4|
|  00| 昭和|    20|  全国|   1945|  1)| 71998104|  33894059|        null|         5|
|  00| 昭和|    15|  全国|   1940|null| 73114308|  36566010|        null|         6|
|  00| 昭和|    25|  全国|   1950|null| 84114574|  41241192|        null|         7|
|  00| 昭和|    30|  全国|   1955|null| 90076594|  44242657|        null|         8|
|  00| 昭和|    35|  全国|   1960|  2)| 94301623|  46300445|        null|         9|
|  00| 昭和|    40

# LAG/LEAD関数
Lagというのはひとつ前のレコード
LEADというのは一つ先のレコード

前日比とか前年比といった様に前後のレコードを比較することによって意味をなす値を出したいときに使います。

今回は、先ほど並び替えたデータを使って、どれくらい人口が増えていったのか確認してみましょう。

In [39]:
from pyspark.sql.window import Window

# window 関数
# window->{paritionBy(=GroupBy)->(order by) -> rank(順番付け)}
# 最後にwindowをoverにいれると結果が出ます

window_schema = Window.partitionBy(analysis_df.code).orderBy(analysis_df.sokei.cast("Long").asc())

analysis_df.withColumn("sokei_per",F.lag("sokei",1).over(window_schema)).show(n=50)
percent=analysis_df.withColumn("sokei_per",F.lag("sokei",1).over(window_schema))

+----+-----+------+------+-------+----+---------+----------+------------+---------+
|code|gengo|wareki|kenmei|seireki| chu|    sokei|jinko_male|jinko_femail|sokei_per|
+----+-----+------+------+-------+----+---------+----------+------------+---------+
|  00| 大正|     9|  全国|   1920|null| 55963053|  28044185|        null|     null|
|  00| 大正|    14|  全国|   1925|null| 59736822|  30013109|        null| 55963053|
|  00| 昭和|     5|  全国|   1930|null| 64450005|  32390155|        null| 59736822|
|  00| 昭和|    10|  全国|   1935|null| 69254148|  34734133|        null| 64450005|
|  00| 昭和|    20|  全国|   1945|  1)| 71998104|  33894059|        null| 69254148|
|  00| 昭和|    15|  全国|   1940|null| 73114308|  36566010|        null| 71998104|
|  00| 昭和|    25|  全国|   1950|null| 84114574|  41241192|        null| 73114308|
|  00| 昭和|    30|  全国|   1955|null| 90076594|  44242657|        null| 84114574|
|  00| 昭和|    35|  全国|   1960|  2)| 94301623|  46300445|        null| 90076594|
|  00| 昭和|    40|  全国|   196

In [40]:
# 順番が上がるごとにどれだけ増えていったかみていきましょう

percent.withColumn("percent",percent.sokei/percent.sokei_per).show()

+----+-----+------+------+-------+----+---------+----------+------------+---------+------------------+
|code|gengo|wareki|kenmei|seireki| chu|    sokei|jinko_male|jinko_femail|sokei_per|           percent|
+----+-----+------+------+-------+----+---------+----------+------------+---------+------------------+
|  00| 大正|     9|  全国|   1920|null| 55963053|  28044185|        null|     null|              null|
|  00| 大正|    14|  全国|   1925|null| 59736822|  30013109|        null| 55963053|1.0674332224155105|
|  00| 昭和|     5|  全国|   1930|null| 64450005|  32390155|        null| 59736822|1.0788991252330096|
|  00| 昭和|    10|  全国|   1935|null| 69254148|  34734133|        null| 64450005| 1.074540614853327|
|  00| 昭和|    20|  全国|   1945|  1)| 71998104|  33894059|        null| 69254148| 1.039621540070062|
|  00| 昭和|    15|  全国|   1940|null| 73114308|  36566010|        null| 71998104|1.0155032415853618|
|  00| 昭和|    25|  全国|   1950|null| 84114574|  41241192|        null| 73114308|1.1504529865754867

In [41]:
# SQLで表現してみましょう

spark.sql("""

select 
*,
sokei / LAG(sokei,1) OVER (PARTITION BY code ORDER BY cast(sokei as long) asc)  as sokei_rank
from super_crush_course.parquet_table_with_no_partition

""").show(n=50)

+----+-----+------+------+-------+----+---------+----------+------------+------------------+
|code|gengo|wareki|kenmei|seireki| chu|    sokei|jinko_male|jinko_femail|        sokei_rank|
+----+-----+------+------+-------+----+---------+----------+------------+------------------+
|  00| 大正|     9|  全国|   1920|null| 55963053|  28044185|        null|              null|
|  00| 大正|    14|  全国|   1925|null| 59736822|  30013109|        null|1.0674332224155105|
|  00| 昭和|     5|  全国|   1930|null| 64450005|  32390155|        null|1.0788991252330096|
|  00| 昭和|    10|  全国|   1935|null| 69254148|  34734133|        null| 1.074540614853327|
|  00| 昭和|    20|  全国|   1945|  1)| 71998104|  33894059|        null| 1.039621540070062|
|  00| 昭和|    15|  全国|   1940|null| 73114308|  36566010|        null|1.0155032415853618|
|  00| 昭和|    25|  全国|   1950|null| 84114574|  41241192|        null|1.1504529865754867|
|  00| 昭和|    30|  全国|   1955|null| 90076594|  44242657|        null|1.0708797502796603|
|  00| 昭和

# ピボットテーブル(と転置(transpose))
ピボットテーブルは簡単にいうとテーブル内の項目をまとめて処理(合計、平均)することです。

一般にデータベースに格納されたテーブルは

- コーヒ　1 100 2022/11/11
- コーヒ　1 100 2022/11/12

といった様にレコード単位で格納されています。
それらのデータを集計して表現してくれるものがピボットテーブル

GroupByしたりすればできるのでは？
と思われるかもしれませんが、pivotテーブルは集計のグループをヘッダーとして持つことができます。
※うまくいえなかったので実際に見てみましょう。

transposeについては次のチャプターにて紹介を行います。

In [42]:
# 各元号の中で一番sokeiが多かったものを取り出している
analysis_df.withColumn("sokei",analysis_df.sokei.cast("long")).groupBy(analysis_df.kenmei).pivot("gengo").max("sokei").show()

+----------------------+----+-------+--------+--------+
|                kenmei|null|   大正|    平成|    昭和|
+----------------------+----+-------+--------+--------+
|人口集中地区以外の地区|null|   null|45458715|48668286|
|                佐賀県|null| 684831|  884316|  973749|
|                栃木県|null|1090428| 2016631| 1866066|
|                京都府|null|1406382| 2647660| 2586574|
|                香川県|null| 700308| 1027006| 1022569|
|                愛媛県|null|1096366| 1515025| 1540628|
|                秋田県|null| 936408| 1227478| 1348871|
|                広島県|null|1617680| 2881748| 2819200|
|                  null|null|   null|    null|    null|
|                宮崎県|null| 691094| 1175819| 1175543|
|              鹿児島県|null|1472193| 1797824| 2044112|
|                埼玉県|null|1394461| 7266534| 5863678|
|                三重県|null|1107692| 1866963| 1747311|
|                島根県|null| 722402|  781021|  929066|
|                徳島県|null| 689814|  832427|  878511|
|                岐阜県|null|1132557| 2107700| 20285

# データフレーム限定 RDDによる一行づつの操作

出番がある様な、無い様な操作ですが一つSparkの特徴であるRDD(低レベル操作)についてみていきましょう。  
Sparkは全てのDataFrameは実行されるときにRDDに変換されて実行されます(そのときに最適なRDD操作に変換してくれる)。

あまり普段RDDを意識することなく操作を可能です。

一応RDDでの操作も見ておきましょう。

RDDに変換すると、mapとかlambdaなどPythonの関数が適用になりますが、最適化をやってくれなくなるのであまりおすすめはできないです。

In [43]:
rdds=json_df.rdd.map(lambda x: len(x.code))
rdds.take(10)

rdds.reduce(lambda a,b: a+b)

1960

In [44]:
spark.stop()

Fri Jan 07 05:31:19 UTC 2022 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for server certificate verification.
Fri Jan 07 05:31:20 UTC 2022 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for s